In [80]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import nltk 
from nltk.corpus import stopwords
import collections
import sys
import codecs
n_palabras_comunes = 10
candidates = ['francisco','pablo','calixto','rodolfo',]
pd.set_option("max_rows", None)

In [2]:
stop_words = set(stopwords.words('spanish'))

In [3]:
def write_file(route, transcription, name):
    file = codecs.open(f'{route}/Transcripcion/{name}.txt',"w","utf-8")#write mode 
    file.write(transcription) 
    file.close() 

In [4]:
def load_data(route):
    entities = pd.read_json(route)
    data = pd.DataFrame([[i["Text"], i["Type"], i["Score"]] for i in entities["Entities"]]
                            , columns=['Entitie', 'Type', 'Score'])
    return data.sort_values('Score', ascending=False).reset_index().drop(columns=['index'])

In [5]:
def load_transcripcion(facultad):
    files = [pd.read_json(f'{facultad}/Subtitulos/{name}.json', encoding='utf-8') for name in candidates]
    transcription = [sub.loc['transcripts']['results'][0]['transcript'] for sub in files]
#     for i in range(len(transcription)):
#         write_file(facultad, transcription[i],candidates[i])
    return pd.DataFrame(transcription, index= candidates, columns= ['transcription'])

In [6]:
def load_discurs(data):
    text = pd.DataFrame(data.lower().replace('.', "").replace(',',"").split(' '))
    return text

In [7]:
def process_most_common(df, name, n):
    return collections.Counter(df.loc[name, 'valid_words']).most_common(n)

In [8]:
def valid_words(df):
    tokens = tokenize_column(df, 'transcription')
    valid_words = pd.DataFrame([len(tokens[i]) for i in range(len(candidates))], index=candidates, columns=['valid_words'])
    return valid_words

In [9]:
def tabulate_common(df,n):
    df['valid_words'] = tokenize_column(df, 'transcription')
    most_common = [process_most_common(df, name, n) for name in candidates]
    most_common =  pd.DataFrame([i for i in most_common ])
    most_common= most_common.transpose().rename(columns={ i:candidates[i] for i in range(4)})
    return most_common

In [10]:
def tokenize_column(df, column_name):
    return(df
            .dropna()
            .apply(lambda row: nltk.word_tokenize(row[column_name]), axis=1)
            .apply(lambda tokens: list(filter(lambda token: token.isalpha(),tokens)))
            .apply(lambda tokens: list(map(lambda token: token.lower(), tokens)))
            .apply(lambda word_list: list(filter(lambda word: word not in stop_words, word_list)))       
    )


In [11]:
def load_sentiment(route):
    all_sentiment = [pd.read_json(f'{route}/Analisis_de_Sentimiento/{name}.json') for name in candidates]
    columns=['Mixed','Negative','Neutral','Positive']
    sentiment = pd.DataFrame([], columns=columns, index=candidates)
    for i in range(len(all_sentiment)):
        temp = pd.DataFrame([])
        for g in range(all_sentiment[i]['Result'].size):
            temp = temp.append([all_sentiment[i]['Result'][g]['SentimentScore']],  ignore_index=True)
        sentiment.loc[candidates[i]] = temp.mean()
    return sentiment

# Facultad de Ingeniería

In [12]:
transcription = load_transcripcion('Ingenieria')

In [13]:
palabras_utiles = valid_words(transcription)
palabras_comunes = tabulate_common(transcription, n_palabras_comunes)
sentiment = load_sentiment('Ingenieria')

In [14]:
palabras_utiles

,valid_words
francisco,1827
pablo,2913
calixto,1847
rodolfo,2171


In [15]:
palabras_comunes

,francisco,pablo,calixto,rodolfo
0,"(universidad, 60)","(universidad, 47)","(universidad, 90)","(universidad, 60)"
1,"(ingeniería, 25)","(facultad, 28)","(estudiantes, 26)","(ser, 22)"
2,"(hoy, 17)","(ingeniería, 24)","(calidad, 21)","(hacer, 21)"
3,"(estudiantes, 16)","(estudiantes, 23)","(rector, 19)","(tener, 21)"
4,"(supuesto, 16)","(profesores, 20)","(hacer, 19)","(magdalena, 18)"
5,"(programas, 15)","(dos, 19)","(propuesta, 15)","(van, 17)"
6,"(posibilidad, 15)","(hoy, 18)","(magdalena, 15)","(vamos, 16)"
7,"(ejemplo, 13)","(profesor, 18)","(recursos, 15)","(poder, 15)"
8,"(acreditación, 13)","(programa, 18)","(educativa, 13)","(momento, 13)"
9,"(tema, 13)","(mil, 18)","(unidos, 12)","(importante, 12)"


In [16]:
sentiment

,Mixed,Negative,Neutral,Positive
francisco,0.125788,0.0170564,0.614211,0.242945
pablo,0.0716548,0.0559598,0.531929,0.340457
calixto,0.0140588,0.0196071,0.652026,0.314308
rodolfo,0.0145522,0.0852771,0.586234,0.313937


# Facultad de Ciencias de la Salud

In [17]:
transcription = load_transcripcion('Salud')

In [18]:
palabras_utiles = valid_words(transcription)
palabras_comunes = tabulate_common(transcription, n_palabras_comunes)
sentiment = load_sentiment('Salud')

In [19]:
palabras_utiles

,valid_words
francisco,1752
pablo,2838
calixto,2110
rodolfo,2117


In [20]:
palabras_comunes

,francisco,pablo,calixto,rodolfo
0,"(universidad, 37)","(universidad, 51)","(universidad, 95)","(universidad, 43)"
1,"(supuesto, 32)","(estudiantes, 36)","(estudiantes, 31)","(ser, 21)"
2,"(estudiantes, 28)","(salud, 32)","(hacer, 24)","(hacer, 18)"
3,"(programa, 21)","(si, 25)","(calidad, 19)","(magdalena, 17)"
4,"(ejemplo, 21)","(mil, 22)","(voy, 14)","(vamos, 15)"
5,"(posibilidad, 17)","(dos, 20)","(si, 14)","(proyecto, 14)"
6,"(institución, 16)","(profesores, 18)","(rector, 14)","(nivel, 14)"
7,"(acreditación, 15)","(programas, 18)","(profesores, 13)","(hecho, 13)"
8,"(hoy, 15)","(hoy, 18)","(trabajar, 13)","(salud, 13)"
9,"(formación, 15)","(programa, 17)","(programas, 12)","(facultad, 12)"


In [21]:
sentiment

,Mixed,Negative,Neutral,Positive
francisco,0.00451146,0.0715566,0.706366,0.217566
pablo,0.0313051,0.0642491,0.617822,0.286623
calixto,0.0169883,0.0197693,0.498912,0.46433
rodolfo,3.34195e-05,0.00240752,0.769474,0.228085


# Facultad de Humanidades

In [22]:
transcription = load_transcripcion('Humanidades')

In [23]:
palabras_comunes = tabulate_common(transcription, n_palabras_comunes )
palabras_utiles = valid_words(transcription)
sentiment = load_sentiment('Humanidades')

In [24]:
palabras_utiles

,valid_words
francisco,1851
pablo,2577
calixto,2041
rodolfo,2416


In [25]:
palabras_comunes

,francisco,pablo,calixto,rodolfo
0,"(investigación, 27)","(universidad, 40)","(universidad, 83)","(universidad, 61)"
1,"(programa, 27)","(derecho, 28)","(rector, 21)","(tener, 23)"
2,"(estudiantes, 25)","(estudiantes, 27)","(propuesta, 17)","(estudiantes, 22)"
3,"(supuesto, 25)","(hoy, 23)","(entonces, 16)","(hacer, 21)"
4,"(universidad, 24)","(hacer, 17)","(facultad, 15)","(magdalena, 19)"
5,"(tema, 20)","(ser, 15)","(va, 15)","(vamos, 19)"
6,"(posibilidad, 19)","(dos, 15)","(estudiantes, 14)","(importante, 17)"
7,"(puedan, 17)","(profesores, 14)","(hacer, 13)","(ser, 17)"
8,"(hacer, 14)","(cuatro, 14)","(ser, 13)","(nivel, 17)"
9,"(tener, 14)","(cine, 14)","(aquí, 12)","(van, 15)"


In [26]:
sentiment

,Mixed,Negative,Neutral,Positive
francisco,0.0601643,0.0394242,0.574009,0.326403
pablo,0.202079,0.00787378,0.458985,0.331062
calixto,0.143777,0.160556,0.369839,0.325829
rodolfo,0.00112164,0.00819304,0.642435,0.34825
